In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import time

### The Task - To Analyze and Simplify a Commercial Loan Dataset
The dataset comes from a loan company, and has some 421,000 rows, and 111 columns. We will try to reduce this number of variables using the Random Forest Classifier and the Principal Components Analysis (PCA).

### Import and Initial Data Manipulation

In [7]:
y2015 = pd.read_csv('C://Users//fergu//LoanStats3d_2.csv', skiprows=1)

C:\Users\fergu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,...,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,...,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,...,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,...,0.0,0.0,91.7,22.2,0.0,0.0,304003.0,74920.0,41500.0,42503.0
4,68495092,73384866.0,8650.0,8650.0,8650.0,36 months,19.89%,320.99,E,E3,...,0.0,12.0,100.0,50.0,1.0,0.0,38998.0,18926.0,2750.0,18248.0


In [9]:
y2015.shape

(421097, 111)

In [10]:
# Will give memory error because of the sheer number of different
# distinct data values in the table for some of the columns  

#from sklearn import ensemble
#from sklearn.model_selection import cross_val_score
#
#randforestclass = ensemble.RandomForestClassifier()
#X = y2015.drop('loan_status', axis=1)
#Y = y2015['loan_status']
#X = pd.get_dummies(X)
#X.head()

In [11]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [12]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421092,36271333.0,38982739.0,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,0.0,3.0,100.0,50.0,1.0,0.0,51239.0,34178.0,10600.0,33239.0
421093,36490806.0,39222577.0,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,1.0,2.0,95.0,66.7,0.0,0.0,96919.0,58418.0,9700.0,69919.0
421094,36271262.0,38982659.0,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,0.0,1.0,100.0,50.0,0.0,1.0,43740.0,33307.0,41700.0,0.0
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
y2015 = y2015[:-2]

### Isolate the Numerical Variables

In [14]:
y2015_num = y2015._get_numeric_data()

In [15]:
y2015_cat = y2015.drop(y2015_num.columns, axis=1)

In [16]:
test = y2015_num.dropna(how='any', axis=1)

### Scale the Numerical Variables

In [17]:
from sklearn.preprocessing import StandardScaler
y2015_num_scaled = StandardScaler().fit_transform(test)
y2015_num_scaled

array([[  1.48578579,   1.50710837,   0.08863448, ...,  -0.42573693,
         11.87443646,  -0.68449758],
       [  1.52223613,   1.54425274,  -0.65804201, ...,   0.06449349,
         -0.43306133,   0.73177985],
       [  1.53407024,   1.55532575,   1.13864831, ...,   0.32265   ,
          0.75281736,   0.87929026],
       ..., 
       [ -1.86373287,  -1.83676857,  -0.26137012, ...,  -0.36768279,
         -0.52005629,  -0.23302288],
       [ -1.8405705 ,  -1.81310089,  -0.37803833, ...,   0.12625625,
         -0.56126443,   0.60170125],
       [ -1.86374037,  -1.83677647,   0.55530729, ...,  -0.38543118,
          0.90391387,  -0.98944042]])

In [18]:
# Convert the numerical variables array to a dataframe
y2015_num_scaled = pd.DataFrame(y2015_num_scaled, columns = test.columns)
y2015_num_scaled.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1.485786,1.507108,0.088634,0.088634,0.089385,0.520931,-0.255135,-0.391692,0.594963,-0.374473,...,-0.05947,-0.180425,-0.092212,-1.692806,-0.345071,4.356849,0.709806,-0.425737,11.874436,-0.684498
1,1.522236,1.544253,-0.658042,-0.658042,-0.657587,-1.183385,-0.585173,-0.229420,0.138019,-0.374473,...,-0.05947,-0.180425,-0.092212,0.684554,-0.345071,-0.145320,-0.482558,0.064493,-0.433061,0.731780
2,1.534070,1.555326,1.138648,1.138648,1.139814,-1.183385,1.370997,0.433191,0.290334,-0.374473,...,-0.05947,-0.180425,-1.171044,0.684554,-0.345071,-0.145320,1.139425,0.322650,0.752817,0.879290
3,1.534075,1.555330,1.488653,1.488653,1.489957,-1.414950,1.699769,0.203305,0.102281,-0.374473,...,-0.05947,-0.180425,-1.171044,-0.250616,-0.345071,-0.145320,0.743408,0.462518,0.894757,-0.022203
4,1.537044,1.558100,-0.768877,-0.768877,-0.768466,1.688017,-0.493648,-0.297034,0.267785,-0.374473,...,-0.05947,-0.180425,5.301951,0.684554,2.245775,-0.145320,-0.765113,-0.678473,-0.879483,-0.574172


In [19]:
y2015_final = y2015_num_scaled.join(y2015_cat)

In [20]:
y2015_final.shape

(421095, 74)

### Define the Input and Output Dataframes

In [21]:
X = y2015_final.drop('loan_status', axis=1)
X = pd.get_dummies(X)
X = X.dropna(axis=1)
X.shape

(421095, 202)

In [22]:
Y = y2015_final['loan_status']

### Define Function to Calculate Random Forest Classification and Cross-Validation Matrix

In [23]:
def crossval(input_df, target, no_folds):
    start_time = time.time()
    from sklearn import ensemble
    from sklearn.model_selection import cross_val_score
    rand_forest_class = ensemble.RandomForestClassifier()
    cvs = cross_val_score(rand_forest_class, input_df, target, cv=no_folds)
    print(cvs)    
    print('Time taken: {} seconds.'.format('%.3f' % (time.time() - start_time)))
    print('Average accuracy: {}'.format('%.3f' % cvs.mean()))

### Calculate RFC, Cross-Validation Matrix for Initial Case - 202 Variables

In [24]:
crossval(X, Y, 10)

[ 0.97708437  0.98069388  0.98162     0.98164375  0.97480408  0.97727381
  0.96627799  0.98076421  0.97990833  0.98014535]
Time taken: 396.929 seconds.
Average accuracy: 0.978


### Define Function to Perform PCA and Calculate RFC, Cross-Validation Matrix

In [25]:
def PCA_and_crossval(input_df, n, target, no_folds):
    start_time = time.time()
    from sklearn.decomposition import PCA
    sklearn_pca = PCA(n_components=n)
    
    X_sklearn = sklearn_pca.fit_transform(input_df)
    print('The percentage of total variance in the dataset explained by each component from SKlearn PCA.\n',
      sklearn_pca.explained_variance_ratio_)
    print('Time taken for PCA transform: {} seconds'.format('%.3f' % (time.time() - start_time)))
    
    start_time = time.time()
    from sklearn import ensemble
    from sklearn.model_selection import cross_val_score
    rand_forest_class = ensemble.RandomForestClassifier()
    cvs = cross_val_score(rand_forest_class, X_sklearn, target, cv=no_folds)
    print(cvs)
    print('Time taken for cross validation: {} seconds.'.format('%.3f' % (time.time() - start_time)))
    print('Average Accuracy: {}'.format(cvs.mean()))

### Calculate RF Classifier and CV Matrix for Test Case - 20 Components


In [26]:
PCA_and_crossval(X, 20, Y, 10)

The percentage of total variance in the dataset explained by each component from SKlearn PCA.
 [ 0.16073683  0.09389221  0.0585092   0.05573897  0.04958622  0.03681615
  0.03403124  0.03245525  0.02979733  0.02796155  0.02753993  0.02107831
  0.01931404  0.01726475  0.01703306  0.01586904  0.01551264  0.01541068
  0.01513336  0.01454239]
Time taken for PCA transform: 21.673 seconds
[ 0.89807889  0.90406307  0.91161454  0.91512906  0.91056756  0.90726668
  0.90448598  0.90819065  0.91578597  0.85512754]
Time taken for cross validation: 509.012 seconds.
Average Accuracy: 0.9030309942035697


### Calculate RF Classifier and CV Matrix for Test Case - 50 Components

In [27]:
PCA_and_crossval(X, 50, Y, 10)

The percentage of total variance in the dataset explained by each component from SKlearn PCA.
 [ 0.16073683  0.09389221  0.0585092   0.05573897  0.04958622  0.03681615
  0.03403124  0.03245525  0.02979733  0.02796155  0.02753993  0.02107831
  0.01931409  0.01726484  0.01703307  0.01586988  0.01551281  0.01541123
  0.01513415  0.01454253  0.0134706   0.01312234  0.01308022  0.01158762
  0.01120916  0.01107309  0.0103477   0.00960519  0.00827796  0.00765462
  0.00729971  0.00701053  0.00596248  0.00564765  0.0053432   0.00507199
  0.00442489  0.00427358  0.00413887  0.00376141  0.00369982  0.00345983
  0.00330974  0.00325521  0.00308103  0.00302739  0.00282645  0.00261204
  0.00233453  0.00227152]
Time taken for PCA transform: 17.115 seconds
[ 0.96031916  0.96687326  0.96986536  0.97026905  0.97052957  0.97195441
  0.97309364  0.96953145  0.96658513  0.95922196]
Time taken for cross validation: 1145.174 seconds.
Average Accuracy: 0.9678242981275347


### Calculate RF Classifier and CV Matrix for Test Case - 10 Components

In [28]:
PCA_and_crossval(X, 10, Y, 10)

The percentage of total variance in the dataset explained by each component from SKlearn PCA.
 [ 0.16073683  0.09389221  0.0585092   0.05573897  0.04958622  0.03681615
  0.03403122  0.03245524  0.02979731  0.02796143]
Time taken for PCA transform: 9.001 seconds
[ 0.90116597  0.91845361  0.92163568  0.922918    0.91688435  0.91308478
  0.90529341  0.90477095  0.91858836  0.84814516]
Time taken for cross validation: 370.136 seconds.
Average Accuracy: 0.9070940257345164


### Try a Correlation Matrix

In [30]:
y2015_final_dummies = pd.get_dummies(y2015_final)

In [31]:
corrmat = y2015_final_dummies.corr()

In [32]:
corrmat.shape

(209, 209)

In [33]:
corrmat.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
id,1.000000,0.997540,-0.008288,-0.008288,-0.008554,-0.053402,-0.012919,0.015055,0.005692,0.002300,...,-0.008433,-0.027861,-0.066300,-0.034040,-0.011424,-0.042871,0.042871,0.031699,0.015205,0.024511
member_id,0.997540,1.000000,-0.008620,-0.008620,-0.008885,-0.053447,-0.013274,0.014958,0.005806,0.002261,...,-0.008466,-0.027835,-0.066006,-0.033964,-0.011364,-0.043300,0.043300,0.032002,0.015380,0.024761
loan_amnt,-0.008288,-0.008620,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,...,0.003849,-0.000398,0.009984,0.000568,-0.004610,-0.016880,0.016880,0.008359,0.007089,0.014350
funded_amnt,-0.008288,-0.008620,1.000000,1.000000,0.999994,0.140572,0.941205,0.305734,0.006103,-0.010424,...,0.003849,-0.000398,0.009984,0.000568,-0.004610,-0.016880,0.016880,0.008359,0.007089,0.014350
funded_amnt_inv,-0.008554,-0.008885,0.999994,0.999994,1.000000,0.140209,0.941187,0.305803,0.006044,-0.010448,...,0.003850,-0.000394,0.009988,0.000578,-0.004605,-0.016832,0.016832,0.008338,0.007071,0.014305


### Isolate all Variables Related to the Outcome - i.e. Loan Status

In [34]:
loan_statuses = ['loan_status_Fully Paid', 'loan_status_Current', 'loan_status_Charged Off', 'loan_status_Late (31-120 days)', 'loan_status_In Grace Period', 'loan_status_Default', 'loan_status_Late (16-30 days)']

In [35]:
corrmat_stat = corrmat[loan_statuses]
corrmat_stat.head()

,loan_status_Fully Paid,loan_status_Current,loan_status_Charged Off,loan_status_Late (31-120 days),loan_status_In Grace Period,loan_status_Default,loan_status_Late (16-30 days)
id,-0.119724,0.146429,-0.083406,0.008371,-0.000731,0.004373,0.005020
member_id,-0.119457,0.146156,-0.083382,0.008412,-0.000748,0.004429,0.005096
loan_amnt,-0.033211,0.015382,0.011895,0.009339,0.011779,0.001988,0.009891
funded_amnt,-0.033211,0.015382,0.011895,0.009339,0.011779,0.001988,0.009891
funded_amnt_inv,-0.033178,0.015401,0.011844,0.009298,0.011769,0.001965,0.009872


In [36]:
# Test whether this command will give the expected output:
# The sum of all values across row 0
sum(corrmat_stat.iloc[0])

-0.039670003942653194

In [37]:
# Create entire new column of the algebraic sum of each row
corrmat_stat['agg'] = [sum(corrmat_stat.iloc[j]) for j in range(0, len(corrmat_stat))]
corrmat_stat.head()

C:\Users\fergu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,loan_status_Fully Paid,loan_status_Current,loan_status_Charged Off,loan_status_Late (31-120 days),loan_status_In Grace Period,loan_status_Default,loan_status_Late (16-30 days),agg
id,-0.119724,0.146429,-0.083406,0.008371,-0.000731,0.004373,0.005020,-0.039670
member_id,-0.119457,0.146156,-0.083382,0.008412,-0.000748,0.004429,0.005096,-0.039494
loan_amnt,-0.033211,0.015382,0.011895,0.009339,0.011779,0.001988,0.009891,0.027063
funded_amnt,-0.033211,0.015382,0.011895,0.009339,0.011779,0.001988,0.009891,0.027063
funded_amnt_inv,-0.033178,0.015401,0.011844,0.009298,0.011769,0.001965,0.009872,0.026972


In [38]:
# Sort the new column in descending order
corrmat_stat['agg'].sort_values(ascending=False)

loan_status_Default                 0.887982
loan_status_Late (16-30 days)       0.828841
loan_status_In Grace Period         0.743873
loan_status_Late (31-120 days)      0.625224
loan_status_Charged Off             0.360243
last_pymnt_d_Aug-2016               0.265382
last_pymnt_d_Dec-2016               0.228519
int_rate                            0.227989
last_pymnt_d_Sep-2016               0.217299
last_pymnt_d_Nov-2016               0.182027
total_rec_late_fee                  0.176385
last_pymnt_d_Oct-2016               0.175733
collection_recovery_fee             0.167451
recoveries                          0.166772
last_credit_pull_d_Oct-2016         0.122148
acc_open_past_24mths                0.119297
num_tl_op_past_12m                  0.107263
grade_E                             0.106415
grade_F                             0.093645
grade_D                             0.091356
inq_last_6mths                      0.085546
verification_status_Verified        0.069009
term_ 60 m

### Select the Top 20 Variables in Terms of Influence on Loan Status
Choose the top 20 variables that correlate positively with the variables denoting loan status. Ignore any variables that have loan status in their names.

In [39]:
newvars20 = ['last_pymnt_d_Aug-2016', 'last_pymnt_d_Dec-2016', 'int_rate', 'last_pymnt_d_Sep-2016', 'last_pymnt_d_Nov-2016', 'total_rec_late_fee',         
'last_pymnt_d_Oct-2016', 'collection_recovery_fee', 'recoveries', 'last_credit_pull_d_Oct-2016', 'acc_open_past_24mths', 'num_tl_op_past_12m',         
'grade_E', 'grade_F', 'grade_D', 'inq_last_6mths', 'verification_status_Verified', 'term_ 60 months', 'last_pymnt_d_Jun-2016', 'last_pymnt_d_Jul-2016']               

In [40]:
X_20vars = X[newvars20]

### Results for Top 20 Positively Correlated Variables
This approach did not give very good accuracy, only 74.8%. No better than the PCA method.

In [41]:
crossval(X_20vars, Y, 10)

[ 0.77955879  0.76457458  0.77694664  0.77716036  0.768891    0.77107575
  0.76380346  0.75413807  0.74997031  0.57383746]
Time taken: 80.992 seconds.
Average accuracy: 0.748


### Select the Top 22 NEGATIVELY CORRELATED Variables in Terms of Influence on Loan Status
This time, we choose the top 22 variables that correlate negatively with the variables that denote loan status, again ignoring any variables that have loan status in their names.

In [42]:
newvars22_neg = ['tot_cur_bal', 'avg_cur_bal', 'home_ownership_MORTGAGE', 'initial_list_status_w', 'mort_acc', 'tot_hi_cred_lim', 
                 'title_Credit card refinancing', 'purpose_credit_card', 'last_credit_pull_d_Jan-2017', 'total_rev_hi_lim', 
                 'last_pymnt_amnt', 'term_ 36 months', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_tl', 
                 'verification_status_Not Verified', 'total_bc_limit', 'grade_B', 'total_pymnt', 
                 'total_pymnt_inv', 'next_pymnt_d_Feb-2017', 'total_rec_prncp', 'grade_A', 'last_pymnt_d_Jan-2017']

In [43]:
X_22vars_neg = X[newvars22_neg]

### Results for Top 22 Negatively Correlated Variables
This time the results were surprisingly accurate, at 94.5%. The accuracy is the highest so far.

In [44]:
crossval(X_22vars_neg, Y, 10)

[ 0.95198404  0.94032438  0.93859087  0.95386004  0.94013299  0.9449062
  0.94706595  0.94428744  0.94141117  0.94430722]
Time taken: 173.120 seconds.
Average accuracy: 0.945


### Select the Same NEGATIVELY CORRELATED Variables but Remove Those Relating to Outstanding Balance or Payment Amount
This time, we are trying to determine whether +90% accuracy can be achieved without considering the outstanding principal or payment amount. So we remove all variables that seem to be related to those factors. This is a bit of a guessing game because some of the variable are hard to discern from their abbreviated names.

In [45]:
# Define list without any variables that seem to relate to outstanding principal, payment amount:
# Remove the following variables:
# 'tot_cur_bal', 'avg_cur_bal', 'total_pymnt', 'total_pymnt_inv', 'last_pymnt_amnt', 'total_rec_prncp'
newvars17_neg = ['home_ownership_MORTGAGE', 'initial_list_status_w', 'mort_acc', 'tot_hi_cred_lim', 
                 'title_Credit card refinancing', 'purpose_credit_card', 'last_credit_pull_d_Jan-2017', 
                 'total_rev_hi_lim', 'term_ 36 months', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_old_rev_tl_op', 
                 'mo_sin_rcnt_tl', 'verification_status_Not Verified', 'total_bc_limit', 'grade_B',  
                  'next_pymnt_d_Feb-2017', 'grade_A', 'last_pymnt_d_Jan-2017']

In [46]:
X_17vars_neg = X[newvars17_neg]

### Results for Top Negatively Correlated Variables but with Balance/ Pmt Amount Variables Excluded
Here the results were still better than with the positively correlated variables, at 85.5%, but below the 90% criterion.

In [47]:
crossval(X_17vars_neg, Y, 10)

[ 0.86996272  0.85381492  0.84870936  0.87124504  0.84497744  0.85307528
  0.85848631  0.84715856  0.84878999  0.85828623]
Time taken: 121.952 seconds.
Average accuracy: 0.855


### Select ONLY Those Variables Relating to Outstanding Balance or Payment Amount
Finally we do the analysis only on the variables that seem to relate to balance, or payment amount.

In [48]:
newvars6_neg = ['tot_cur_bal', 'avg_cur_bal', 'total_pymnt', 'total_pymnt_inv', 'last_pymnt_amnt', 'total_rec_prncp']

In [49]:
X_6vars_neg = X[newvars6_neg]

### Results for Top Negatively Correlated Variables Relating to Outstanding Balance or Payment Amount
The results of this exceed the 90% target threshold. Thus we have successfully reduced a dataset initially containing over 100 variables, to the most important 6 variables, while retaining over 90% accuracy on average. The 76.7%  accuracy in the least accurate fold is, however, a concern, and is evidence of overfitting.

In [50]:
crossval(X_6vars_neg, Y, 10)

[ 0.76162998  0.9394695   0.93863836  0.93410273  0.9275469   0.92417478
  0.92267686  0.92182194  0.921177    0.91870517]
Time taken: 220.817 seconds.
Average accuracy: 0.911
